# Data Analysis 1

In [1]:
#importing libraries
import os
import nltk
import re
import numpy as np
import regex
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# importing cvs files
loose=pd.read_csv('original-sheets\Responses-Loose.csv')
student_info=pd.read_csv('corpus-files\student_information.csv')
course=pd.read_csv('corpus-files\course.csv')
answer_df=pd.read_csv('answer.csv',index_col = 'answer_id', low_memory=False)


In [5]:
#dropping to only relevant columns
loose=loose.drop(columns=['3Langs','Item','gender','course_id','class_id','version','text1_len','text2_len','text1',
                         'text2 (line breaks/extra spaces removed, spaces added to reach 60)','Judgement','Notes'])

,answer_id,anon_id,L1,level_id,question_id,text3_len,text3 (edits made to fix word counts)
0,141.0,aj8,Chinese,5.0,17.0,114.0,"Today, I am going to describe one of my classm..."
1,143.0,az8,Chinese,5.0,17.0,95.0,My niece is 3 years old who is my younger brot...
2,133.0,az2,Korean,5.0,17.0,128.0,"When I was in Germany, I met a friend who was ..."
3,135.0,at8,Korean,5.0,17.0,105.0,my friend is ANON_NAME_0. she is a my ELI frie...
4,188.0,eh9,Korean,5.0,17.0,97.0,Younghun is my best friend. His facial appeara...
...,...,...,...,...,...,...,...
1807,48246.0,bv8,Chinese,4.0,6105.0,131.0,The quote that talks about a true friend is so...
1808,48254.0,dr8,Korean,4.0,6105.0,162.0,I think this means that home is not just a pla...
1809,48252.0,bl3,Spanish,4.0,6105.0,146.0,This quotation means that the majority of peop...
1810,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
course_=course.drop(columns=['class_id','semester','section'])

student=student_info.drop(columns=['birth_year','language_used_at_home','non_native_language_1','yrs_of_study_lang1',
                                  'study_in_classroom_lang1','ways_of_study_lang1','non_native_language_2','study_in_classroom_lang2',
                                  'ways_of_study_lang2','non_native_language_3','yrs_of_study_lang3','study_in_classroom_lang3',
                                  'ways_of_study_lang3','yrs_of_english_learning','yrs_in_english_environment','yrs_of_study_lang2'])
answer_=answer_df[['question_id','anon_id','course_id','version','text_len','text','tokens']].reset_index()

In [8]:
#This function removes all of the rows that contain a string in the passed column
def rmStr (df, col):
    row_index=0
    row_ind=[]
    for i in df[col]:
        try:
            int(i)
        except:
            row_ind.append(row_index)
        row_index+=1
    return df.drop(labels=row_ind,axis=0)

In [9]:
#Cleaning the answer df
col='course_id'
answer=rmStr(answer_, col) #removes strings from answer_.course_id 

In [32]:
# Merging the datasets

merge_ans=answer[['anon_id','course_id']] #sets answer df up for merging 
student_merge=student.drop(columns=['course_history']) #set student df up for merging

student_ans=student_merge.merge(answer,on='anon_id').astype({'course_id':'int64'}) #merges student and answers

stu_ans_crs_=student_ans.merge(course_, on='course_id').drop(columns=['age','course_id','version','tokens']) #merges the student-answers df with course df

In [34]:
#drops rows that had intelligible data
index=0
ind_list=[]
for i in stu_ans_crs_.text_len:
    txtlen=int(i)
    if txtlen < 60:
        ind_list.append(index)
    index+=1

stu_ans_crs_=stu_ans_crs_.drop(labels=ind_list,axis=0).reset_index().drop(columns=['index'])
stu_ans_crs_

,anon_id,gender,native_language,answer_id,question_id,text_len,text,level_id
0,do6,Female,Russian,150,4,299,Some people prefer eat out and some like doing...,5
1,do6,Female,Russian,1221,97,288,My opinion is that a person does need educatio...,5
2,do6,Female,Russian,1957,189,321,There are two national rooms in the Cathedral ...,5
3,do6,Female,Russian,2164,190,464,There are two nation rooms in the Cathedral of...,5
4,bv5,Male,Arabic,151,4,315,"""Not all learning takes place in the classroom...",5
...,...,...,...,...,...,...,...,...
17252,bh8,Male,Japanese,48034,6087,100,I received my Medical Doctor license in Japan ...,3
17253,bh8,Male,Japanese,48293,6119,138,I introduce my ideal home to you quickly. This...,3
17254,cz3,Female,Korean,47451,6027,101,I will put 5 items in time capsule such as som...,3
17255,cz3,Female,Korean,48001,6087,100,I have studied Visual Design since 2002. I gra...,3
